# Notebook 12: Reflection-Enforced Agent Graph

Purpose:
- Integrate the Reflection Agent directly into LangGraph
- Enforce post-LLM safety and alignment checks
- Prevent unsafe or overconfident outputs from reaching the user
- Build a self-regulating, production-grade AI agent

This notebook is fully self-contained.

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
import os
from dotenv import load_dotenv
from google import genai

In [2]:
load_dotenv()

GEMINI_API_KEY = os.getenv("Google_Gemini_Api_Key")
if not GEMINI_API_KEY:
    raise ValueError("Gemini API key not found")

client = genai.Client(api_key=GEMINI_API_KEY)

In [3]:
class AgentState(TypedDict):
    profile: dict
    memory: dict
    analytics: dict
    supervisor: dict
    llm_response: str
    reflection: dict

In [4]:
user_profile = {
    "education": "MCA",
    "goal": "Become an ML Engineer",
    "priority": "Learning + Health"
}

In [5]:
analytics_summary = {
    "avg_work_minutes": 421,
    "avg_leisure_minutes": 255,
    "avg_sleep_minutes": 426,
    "avg_exercise_minutes": 64,
    "mean_productivity": 82
}

In [6]:
supervisor_decision = {
    "status": "ALLOW",     # ALLOW | BLOCK
    "confidence": "LOW",   # LOW | MEDIUM | HIGH
    "warnings": [
        "Productivity instability detected",
        "Conflict: high output but reported fatigue"
    ]
}

In [7]:
memory_payload = {
    "short_term": {
        "days_observed": 5,
        "avg_work_minutes": 490,
        "avg_sleep_minutes": 356,
        "avg_exercise_minutes": 18,
        "mean_productivity": 81,
        "productivity_trend": "stable"
    },
    "reflective": "Sleep duration has been consistently low; Physical activity levels are minimal",
    "profile": user_profile
}

In [8]:
def build_reasoning_prompt(profile, analytics, supervisor):
    warnings_text = (
        "\n".join(f"- {w}" for w in supervisor["warnings"])
        if supervisor["warnings"]
        else "None"
    )

    return f"""
You are NEEL, a cautious and responsible AI assistant.

USER CONTEXT:
- Education: {profile['education']}
- Goal: {profile['goal']}
- Priority: {profile['priority']}

ANALYTICS SUMMARY:
- Average Work Time: {analytics['avg_work_minutes']} minutes/day
- Average Leisure Time: {analytics['avg_leisure_minutes']} minutes/day
- Average Sleep Time: {analytics['avg_sleep_minutes']} minutes/day
- Average Exercise Time: {analytics['avg_exercise_minutes']} minutes/day
- Mean Productivity Score: {analytics['mean_productivity']}

SUPERVISOR STATUS:
- Confidence Level: {supervisor['confidence']}
- Warnings:
{warnings_text}

INSTRUCTIONS:
- Do NOT give medical or absolute advice
- Be cautious if confidence is LOW
- Explicitly acknowledge uncertainty
- Do NOT mention ML models or scores
- Keep tone professional and calm

STYLE GUIDELINES:
- At most one subtle emoji may be used if appropriate
- Emojis must be calm and professional (🧠 ⚖️ 🌱 ⏳)
- Emojis are optional

RESPONSE FORMAT:
OBSERVATION:
REASONING:
SUGGESTION:
CONFIDENCE NOTE:
"""

In [9]:
def reflection_agent(llm_text, confidence, user_goal, user_priority):
    issues = []
    text = llm_text.lower()

    absolute_phrases = ["you should", "must", "definitely", "increase your"]
    if any(p in text for p in absolute_phrases):
        issues.append("Uses prescriptive or commanding language")

    if confidence == "LOW" and "should" in text:
        issues.append("Overconfident language for LOW confidence")

    if "increase" in text and "health" in user_priority.lower():
        issues.append("Suggestion may conflict with health priority")

    if not issues:
        return {"decision": "PASS", "issues": []}

    if len(issues) <= 2:
        return {"decision": "SOFTEN", "issues": issues}

    return {"decision": "REJECT", "issues": issues}

In [10]:
def analytics_node(state: AgentState) -> AgentState:
    return state

In [11]:
def supervisor_node(state: AgentState) -> AgentState:
    if state["supervisor"]["status"] != "ALLOW":
        state["llm_response"] = "Reasoning blocked by Supervisor."
    return state

In [12]:
def llm_reasoning_node(state: AgentState) -> AgentState:
    prompt = build_reasoning_prompt(
        state["profile"],
        state["analytics"],
        state["supervisor"]
    )

    response = client.models.generate_content(
        model="models/gemini-2.5-flash",
        contents=prompt
    )

    state["llm_response"] = response.text
    return state

In [13]:
def reflection_node(state: AgentState) -> AgentState:
    result = reflection_agent(
        llm_text=state["llm_response"],
        confidence=state["supervisor"]["confidence"],
        user_goal=state["profile"]["goal"],
        user_priority=state["profile"]["priority"]
    )

    state["reflection"] = result
    return state

In [14]:
def route_after_supervisor(state: AgentState):
    if state["supervisor"]["status"] == "ALLOW":
        return "llm_reasoning"
    return END


def route_after_reflection(state: AgentState):
    decision = state["reflection"]["decision"]

    if decision == "PASS":
        return END

    if decision == "SOFTEN":
        state["llm_response"] = (
            "I may need a bit more context before offering guidance. "
            "Could you share more about your routine?"
        )
        return END

    if decision == "REJECT":
        state["llm_response"] = (
            "I’m not confident enough to provide guidance yet. "
            "Gathering more information would help."
        )
        return END

In [15]:
graph = StateGraph(AgentState)

graph.add_node("analytics", analytics_node)
graph.add_node("supervisor", supervisor_node)
graph.add_node("llm_reasoning", llm_reasoning_node)
graph.add_node("reflection", reflection_node)

graph.set_entry_point("analytics")

graph.add_edge("analytics", "supervisor")

graph.add_conditional_edges(
    "supervisor",
    route_after_supervisor,
    {
        "llm_reasoning": "llm_reasoning",
        END: END
    }
)

graph.add_edge("llm_reasoning", "reflection")

graph.add_conditional_edges(
    "reflection",
    route_after_reflection,
    {END: END}
)

neel_agent = graph.compile()

In [16]:
initial_state = {
    "profile": user_profile,
    "memory": memory_payload,
    "analytics": analytics_summary,
    "supervisor": supervisor_decision,
    "llm_response": "",
    "reflection": {}
}

final_state = neel_agent.invoke(initial_state)

print("FINAL RESPONSE:\n")
print(final_state["llm_response"])

print("\nREFLECTION DECISION:")
print(final_state["reflection"])

FINAL RESPONSE:

OBSERVATION:
Based on the provided information, there is a clear commitment to dedicated work and learning, alongside commendable engagement in physical exercise. However, the supervisor's report highlights a conflict between high output and reported fatigue, accompanied by detected productivity instability. Additionally, the average sleep duration appears to be around the lower end of common recommendations.

REASONING:
It is plausible that the sustained demands of intensive work and learning, while contributing to significant output, might be challenging to maintain consistently over time without optimal recovery. When sleep duration is towards the lower range, it could potentially impact an individual's capacity for full restoration, which in turn might manifest as fatigue and contribute to variations in focus and output. This dynamic could be a factor in the observed instability.

SUGGESTION:
Given your primary goal of becoming an ML Engineer and your priorities of

This agent enforces:

- Pre-reasoning validation (Supervisor)
- Controlled LLM reasoning
- Post-reasoning safety checks (Reflection)
- Deterministic execution flow

NEEL is now a self-regulating AI system.